<a href="https://colab.research.google.com/github/sudhakarmlal/dog-classifier/blob/main/DogClassifier_PytorchLightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.5 MB/s eta 0:00:00


In [ ]:
!pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 16.1 MB/s eta 0:00:00


In [ ]:
import os
import random
import timm
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data import random_split, SubsetRandomSampler
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from lightning.pytorch import LightningModule
from lightning.pytorch import Trainer
import lightning.pytorch as pl
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from PIL import Image
from torch import nn, optim
from pathlib import Path
from typing import Union
from torch.optim.lr_scheduler import MultiStepLR
from lightning.pytorch.utilities import rank_zero_info
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from torchmetrics import Accuracy




In [ ]:
transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(224),             # resize shortest side to 224 pixels
        transforms.CenterCrop(224),         # crop longest side to 224 pixels at center
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])

In [ ]:
import gdown
import zipfile
class DataModule(pl.LightningDataModule):

    def __init__(self, transform=transform, batch_size=32):
        super().__init__()
        self.root_dir = "/content/gdrive/MyDrive/Dog_Breed/exp/dataset"
        self.transform = transform
        self.batch_size = batch_size

    def setup(self, stage=None):
        file_id = "1-lbK2hPkOeZB-RfwLmeBj3oOSbsWU-ov"
        output_file = "dt1.zip"
        gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

#output = "dog-breed-image-dataset_gd_exp2.zip"
#gdown.download(url, output)
        zip_ref = zipfile.ZipFile("dt1.zip", 'r')
        zip_ref.extractall("/content/gdrive/MyDrive/Dog_Breed/exp")
        zip_ref.close()
        dataset = datasets.ImageFolder(root=self.root_dir, transform=self.transform)
        n_data = len(dataset)
        n_train = int(0.8 * n_data)
        n_test = n_data - n_train
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, [n_train, n_test])
        self.train_dataset = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        self.test_dataset = DataLoader(test_dataset, batch_size=self.batch_size)

    def train_dataloader(self):
        return self.train_dataset

    def test_dataloader(self):
        return self.test_dataset

In [ ]:
class ConvolutionalNetwork(LightningModule):

    def __init__(self):
        super(ConvolutionalNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3, 1)
        self.conv2 = nn.Conv2d(6, 16, 3, 1)
        self.fc1 = nn.Linear(16 * 54 * 54, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        self.fc4 = nn.Linear(20, 10)

    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 16 * 54 * 54)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = F.relu(self.fc3(X))
        X = self.fc4(X)
        return F.log_softmax(X, dim=1)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        return optimizer

    def training_step(self, train_batch, batch_idx):
        X, y = train_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def validation_step(self, val_batch, batch_idx):
        X, y = val_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("val_loss", loss)
        self.log("val_acc", acc)

    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        y_hat = self(X)
        loss = F.cross_entropy(y_hat, y)
        pred = y_hat.argmax(dim=1, keepdim=True)
        acc = pred.eq(y.view_as(pred)).sum().item() / y.shape[0]
        self.log("test_loss", loss)
        self.log("test_acc", acc)

In [ ]:
checkpoint_callback = ModelCheckpoint(save_on_train_epoch_end=True,dirpath='/content/gdrive/MyDrive/Dog_Breed',filename="model_tr")

In [ ]:
class DogClassifier(pl.LightningModule):
    def __init__(self, lr: float = 1e-3):
        super().__init__()
        self.lr = lr

        # Load pre-trained ResNet18 model
        self.model = timm.create_model('resnet18', pretrained=True, num_classes=10)

        # Multi-class accuracy with num_classes=2
        self.train_acc = Accuracy(task="multiclass", num_classes=10)
        self.val_acc = Accuracy(task="multiclass", num_classes=10)

        self.save_hyperparameters()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = F.softmax(logits, dim=1)
        self.train_acc(preds, y)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", self.train_acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        preds = F.softmax(logits, dim=1)
        self.val_acc(preds, y)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_acc, prog_bar=True)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.lr)

# Visualization: Model Architecture
def visualize_model(model):
    from torchviz import make_dot
    x = torch.randn(1, 3, 224, 224)
    y = model(x)
    dot = make_dot(y, params=dict(model.named_parameters()))
    dot.render("model_architecture", format="png")
    display(Image("model_architecture.png"))

In [ ]:
if __name__ == '__main__':
    datamodule = DataModule()
    datamodule.setup()
    model = DogClassifier()
    #trainer = pl.Trainer(max_epochs=1)
    trainer = pl.Trainer(
        limit_train_batches=0.05,
        limit_val_batches=0.05,
        max_epochs=1,
        callbacks=[checkpoint_callback],
        log_every_n_steps=10,
        accelerator="auto",
    )
    trainer.fit(model, datamodule)
    datamodule.setup(stage='test')
    test_loader = datamodule.test_dataloader()
    trainer.test(dataloaders=test_loader)

Downloading...
From: https://drive.google.com/uc?id=1-lbK2hPkOeZB-RfwLmeBj3oOSbsWU-ov
To: /content/dt1.zip
100%|██████████| 4.18M/4.18M [00:00<00:00, 228MB/s]
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
Downloading...
From: https://drive.google.com/uc?id=1-lbK2hPkOeZB-RfwLmeBj3oOSbsWU-ov
To: /content/dt1.zip
100%|██████████| 4.18M/4.18M [00:00<00:00, 243MB/s]
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /content/gdrive/MyDrive/Dog_Breed exists and is not empty.
INFO: 
  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | model     | ResNet 

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
Downloading...
From: https://drive.google.com/uc?id=1-lbK2hPkOeZB-RfwLmeBj3oOSbsWU-ov
To: /content/dt1.zip
100%|██████████| 4.18M/4.18M [00:00<00:00, 148MB/s]
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/checkpoint_connector.py:145: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
Downloading...
From: https://drive.google.com/uc?id=1-lbK2hPkOeZB-RfwLmeBj3oOSbsWU-ov
To: /content/dt1.zip
100%|██████████| 4.18M/4.18M [00:00<00:00, 184MB/s]
INFO: Restoring states from the checkpoint path at /content/gdrive/MyDrive/Dog_Breed/model_tr.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states f

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.21134020388126373    │
│         val_loss          │    2.2143173217773438     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
best_model_path = checkpoint_callback.best_model_path
model = DogClassifier.load_from_checkpoint(best_model_path)

In [ ]:
class_names=['Beagle','Bulldog','German_Shepherd','Labrador_Retriever', 'Rottweiler','Boxer','Dachshund','Golden_Retriever','Poodle','Yorkshire_Terrier']
device = torch.device("cpu")   #"cuda:0"
model = DogClassifier().to(device)
    # create model and load state dict
    #model.load_state_dict(torch.load("logs/model_tr.ckpt"))
model.load_state_dict(torch.load("/content/gdrive/MyDrive/Dog_Breed/model_tr.ckpt", map_location=torch.device('cpu'))['state_dict'])
model.eval()
y_true=[]
y_pred=[]
datamodule = DataModule()
datamodule.setup()
with torch.no_grad():
  for test_data in datamodule.test_dataloader():
    test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
    pred = model(test_images).argmax(dim=1)
    for i in range(len(pred)):
      y_true.append(test_labels[i].item())
      y_pred.append(pred[i].item())
print(classification_report(y_true,y_pred,target_names=class_names,digits=4))

<ipython-input-24-1689b40d6c9f>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/gdrive/MyDrive/Dog_Breed/model_tr.ckpt", map_loca

My report
                    precision    recall  f1-score   support

            Beagle     0.0357    0.0455    0.0400        22
           Bulldog     0.1613    0.4545    0.2381        11
   German_Shepherd     0.6667    0.1000    0.1739        20
Labrador_Retriever     1.0000    0.1429    0.2500        14
        Rottweiler     0.0000    0.0000    0.0000        23
             Boxer     1.0000    0.0556    0.1053        18
         Dachshund     0.0000    0.0000    0.0000        21
  Golden_Retriever     0.0000    0.0000    0.0000        23
            Poodle     0.0000    0.0000    0.0000        21
 Yorkshire_Terrier     0.1770    0.9524    0.2985        21

          accuracy                         0.1598       194
         macro avg     0.3041    0.1751    0.1106       194
      weighted avg     0.2660    0.1598    0.0961       194



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
!ls /content/gdrive/MyDrive

Dog_Breed


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Dog_Breed

dataset  exp  gdowndataset  Predictions  Samples


In [ ]:
!ls /content/drive/MyDrive/Dog_Breed/Predictions

 Boxer_100_prediction.txt	       'Labrador Retriever_100_prediction.txt'
 Bulldog_100_prediction.txt	       'Yorkshire Terrier_100_prediction.txt'
'Golden Retriever_100_prediction.txt'


In [ ]:
device = torch.device("cpu")   #"cuda:0"
    #data_module = DogDataModule()
    #model = DogClassifier(lr=1e-3)
model = DogClassifier().to(device)
    # create model and load state dict
    #model.load_state_dict(torch.load("logs/model_tr.ckpt"))
model.load_state_dict(torch.load("/content/gdrive/MyDrive/Dog_Breed/model_tr.ckpt", map_location=torch.device('cpu'))['state_dict'])
out_path='/content/drive/MyDrive/Dog_Breed/Predictions'
input_path='/content/drive/MyDrive/Dog_Breed/Samples'
image_files = [
        f
        for f in os.listdir(input_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ]
class_labels =['Beagle','Bulldog','German_Shepherd','Labrador_Retriever', 'Rottweiler','Boxer','Dachshund','Golden_Retriever','Poodle','Yorkshire_Terrier']
for filename in image_files:
  image_path = os.path.join(input_path, filename)
  #img, predicted_label, confidence = inference(model, image_path)


  transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]
  )

    # Apply the transform to the image
  img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    # Move the input tensor to the same device as the model
  img_tensor = img_tensor.to(model.device)

    # Set the model to evaluation mode
  model.eval()

    # Perform inference
  with torch.no_grad():
    output = model(img_tensor)
    probabilities = F.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()
    # Map the predicted class to the label
    #class_labels = ["cat", "dog"]  # Assuming 0 is cat and 1 is dog

    predicted_label = class_labels[predicted_class]
    confidence = probabilities[0][predicted_class].item()

    output_image_path =os.path.join(out_path
                , f"{os.path.splitext(filename)[0]}_prediction.png"
            )
    output_text_path = os.path.join(
               out_path, f"{os.path.splitext(filename)[0]}_prediction.txt"
            )
    with open(output_text_path, "w") as f:
                f.write(f"Predicted: {predicted_label}\nConfidence: {confidence:.2f}")
    plt.figure(figsize=(8, 8))
    img = Image.open(image_path).convert("RGB")
    plt.imshow(img)
    plt.axis("off")
    plt.title(
        f"Predicted: {predicted_label.capitalize()} (Confidence: {confidence:.2f})"
    )

    # Save the figure
    plt.savefig(output_image_path)
    plt.close()

<ipython-input-48-4d687b0f9e59>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/gdrive/MyDrive/Dog_Breed/model_tr.ckpt", map_loca

In [ ]:
!cp